In [1]:
import pandas as pd
import numpy as np
from functools import reduce
import os

# Basic knowledge: 
To begin this project, it would be good to hold a minimum understanding of the subject we will be analyzing. As I did not know much about this topic at the day the project started, I have recurred to the shark-attack wiki: https://en.wikipedia.org/wiki/Shark_attack



# Defining the dataset path, and importing it to begin basic dataset exploration

Questions: 
- how to add the dataset to gitignore?

In [2]:
# To follow along and access the DataSet, download it from KAGGLE using this link
# https://www.kaggle.com/teajay/global-shark-attacks

# Once you have downloaded the DataSet, change the dataset variable to match the 
# path where you have saved the 'attacks.csv' file.
dataset = 'attacks.csv' 
df = pd.read_csv(dataset, encoding='latin-1')

Now, we will check some basic information about the dataset, in order to formulate a more educated hypothesis which we could actually put to test with the data available.

In [3]:
display(df.shape)# To know the shape of the DF
print(df.drop_duplicates().shape) # Shape when eliminating duplicates

(25723, 24)

(6312, 24)


Here, I notice that the shape of the df with no duplicates is very small when compared to the whole df. This seems weird and since the `drop_duplicates` function ignores time indexes, I'll try to compare both dataframes' time data

In [4]:
df.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22',
       'Unnamed: 23'],
      dtype='object')

In [5]:
df.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
25718            NaN
25719            NaN
25720            NaN
25721            NaN
25722            NaN
Name: Date, Length: 25723, dtype: object

In [6]:
df_nodupes = df.drop_duplicates()
df_nodupes.Date

0        25-Jun-2018
1        18-Jun-2018
2        09-Jun-2018
3        08-Jun-2018
4        04-Jun-2018
            ...     
6307             NaN
6308             NaN
6309             NaN
8702             NaN
25722            NaN
Name: Date, Length: 6312, dtype: object

In [7]:
# Now, with a df smaller in size, I want to see what info is there on the last couple
# of columns which have unexplicit
df_nodupes[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']]

,Case Number.1,Case Number.2,original order,Unnamed: 22,Unnamed: 23
0,2018.06.25,2018.06.25,6303.0,NaN,NaN
1,2018.06.18,2018.06.18,6302.0,NaN,NaN
2,2018.06.09,2018.06.09,6301.0,NaN,NaN
3,2018.06.08,2018.06.08,6300.0,NaN,NaN
4,2018.06.04,2018.06.04,6299.0,NaN,NaN
...,...,...,...,...,...
6307,NaN,NaN,6309.0,NaN,NaN
6308,NaN,NaN,6310.0,NaN,NaN
6309,NaN,NaN,NaN,NaN,NaN
8702,NaN,NaN,NaN,NaN,NaN


In [8]:
# Too many null values... let's count them and let
print(df_nodupes.shape)
df_nodupes[['Case Number.1', 'Case Number.2', 'original order', 'Unnamed: 22', 'Unnamed: 23']].isnull().sum()

(6312, 24)


Case Number.1       10
Case Number.2       10
original order       3
Unnamed: 22       6311
Unnamed: 23       6310
dtype: int64

In [9]:
# If only 1 value in the 'Unnamed: 22' column, and 2 values in the
# 'Unnamed: 22' column, I'll not consider this data for my analysis.
df_nodupes = df_nodupes.drop(columns=['Unnamed: 22', 'Unnamed: 23'])

In [10]:
# Now we'll look at the columns again
df_nodupes.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2', 'original order'],
      dtype='object')

In [18]:
# The following columns seemed a little bit rare, so i do a value count to find out what they are about
df_nodupes['Case Number.1'].value_counts()

1966.12.26      2
1952.08.04      2
1920.00.00.b    2
1913.08.27.R    2
2013.10.05      2
               ..
1955.00.00.a    1
1983.07.12.b    1
1928.11.18      1
1977.10.30      1
2014.06.01.a    1
Name: Case Number.1, Length: 6285, dtype: int64

In [19]:
df_nodupes['Case Number.2'].value_counts()

2012.09.02.b    2
1983.06.15      2
1962.06.11.b    2
1913.08.27.R    2
2006.09.02      2
               ..
1928.11.18      1
1977.10.30      1
2014.10.18      1
1978.12.12      1
2014.06.01.a    1
Name: Case Number.2, Length: 6286, dtype: int64

In [20]:
df_nodupes['original order'].value_counts()

569.0     2
4603.0    1
4899.0    1
810.0     1
796.0     1
         ..
3508.0    1
3256.0    1
3106.0    1
3080.0    1
6272.0    1
Name: original order, Length: 6308, dtype: int64

In [22]:
#Since 'original order seems like arbitrary indexes, i'll drop it
df_nodupes = df_nodupes.drop(columns='original order')
df_nodupes.columns

Index(['Case Number', 'Date', 'Year', 'Type', 'Country', 'Area', 'Location',
       'Activity', 'Name', 'Sex ', 'Age', 'Injury', 'Fatal (Y/N)', 'Time',
       'Species ', 'Investigator or Source', 'pdf', 'href formula', 'href',
       'Case Number.1', 'Case Number.2'],
      dtype='object')

In [29]:
# I want to check out some of the pdf and href
print(df_nodupes[['pdf', 'href']].isnull().sum())
df_nodupes[['pdf', 'href']]

pdf     10
href    10
dtype: int64


,pdf,href
0,2018.06.25-Wolfe.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
1,2018.06.18-McNeely.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
2,2018.06.09-Denges.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
3,2018.06.08-Arrawarra.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
4,2018.06.04-Ramos.pdf,http://sharkattackfile.net/spreadsheets/pdf_di...
...,...,...
6307,NaN,NaN
6308,NaN,NaN
6309,NaN,NaN
8702,NaN,NaN


In [ ]:
display(df.columns) # To know which are the columns in the DF
display(df.count()) # To know how much data are we missin on each column
display(df.dtypes)